In [2]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
from vvasp.VizClasses import NeuropixelsChronicHolder
from vvasp.atlas_utils import Atlas
import pyvista as pv
from labdata.schema import *

[2024-05-20 14:59:29,186][INFO]: Connecting mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306
[2024-05-20 14:59:29,685][INFO]: Connected mmelin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306


In [15]:
cp.ChronicInsertion()

subject_name unique mouse id,procedure_type Defines procedures that are not an experimental session,procedure_datetime,probe_id probe id to keep track or re-uses,holder_id


In [14]:
from labdata.schema import *
from labdata import chronic_paper as cp
cp.ChronicHolderType()

subject = 'MM008'
targets = [['ACA','CP'], ['MOp','CP'], ['VISp']]
holder_ids = [0,0,0]
hemisphere = 'left'

sdict = dict(subject_name=subject,
             procedure_type='chronic implant')
keys = (ProbeInsertion() & sdict).proj().fetch(as_dict=True, order_by='insertion_ap')
for targs, id, key in zip(targets, holder_ids, keys):
    key['holder_id'] = id
    #cp.ChronicInsertion.insert1(key, skip_duplicates=True)
    #cp.ChronicInsertion.delete(key)
    for targ in targs:
        key['region_acronym'] = targ
        key['atlas_id'] = 'allen-ccf'	
        key['hemisphere'] = hemisphere
        #cp.ChronicInsertion.TargetedRegion().delete(key,)
        #cp.ChronicInsertion.TargetedRegion().insert1(key, skip_duplicates=True, ignore_extra_fields=True)


[2024-05-20 15:04:19,659][INFO]: Deleting 3 rows from `paper_2024_chronic_holder`.`chronic_insertion`
[2024-05-20 15:04:22,358][INFO]: Deletes committed.
[2024-05-20 15:04:22,634][INFO]: Deleting 0 rows from `paper_2024_chronic_holder`.`chronic_insertion__targeted_region`
[2024-05-20 15:04:22,635][WARNING]: Nothing to delete.
[2024-05-20 15:04:22,941][INFO]: Deleting 0 rows from `paper_2024_chronic_holder`.`chronic_insertion__targeted_region`
[2024-05-20 15:04:22,942][WARNING]: Nothing to delete.
[2024-05-20 15:04:23,249][INFO]: Deleting 0 rows from `paper_2024_chronic_holder`.`chronic_insertion`
[2024-05-20 15:04:23,250][WARNING]: Nothing to delete.
[2024-05-20 15:04:23,557][INFO]: Deleting 0 rows from `paper_2024_chronic_holder`.`chronic_insertion__targeted_region`
[2024-05-20 15:04:23,558][WARNING]: Nothing to delete.
[2024-05-20 15:04:23,863][INFO]: Deleting 0 rows from `paper_2024_chronic_holder`.`chronic_insertion__targeted_region`
[2024-05-20 15:04:23,864][WARNING]: Nothing to d

In [4]:
cp.ChronicHolderType()
cp.ChronicInsertion().TargetedRegion()
ProbeInsertion()

subject_name unique mouse id,procedure_type Defines procedures that are not an experimental session,procedure_datetime,probe_id probe id to keep track or re-uses,insertion_ap anterior posterior distance from bregma,insertion_ml median lateral distance from bregma,insertion_depth insertion depth (how much shank is inserted from dura),insertion_el elevation of the probe (angle),insertion_az azimuth of the probe (angle),insertion_spin spin on the probe shanks
JC118,chronic implant,2023-09-11 15:30:00,20403312213,-1701.0,-1154.0,3252.0,10.0,-80.0,0.0
JC118,chronic implant,2023-09-11 15:30:00,20403312442,836.0,-981.0,2955.0,70.0,0.0,180.0
JC118,chronic implant,2023-09-11 15:30:00,20403312751,-3687.0,-2823.0,3496.0,125.0,-30.0,0.0
JC118,chronic implant,2023-09-11 15:30:00,20403314292,-2903.0,1428.0,2376.0,150.0,90.0,0.0
JC119,chronic implant,2023-08-08 11:00:00,20403312621,-4221.0,-2339.0,3079.0,135.0,0.0,0.0
JC119,chronic implant,2023-08-08 11:00:00,20403312751,-1701.0,-1154.0,3252.0,90.0,10.0,180.0
JC131,chronic implant,2023-07-19 20:00:00,20403312753,-3761.0,-2886.0,4500.0,120.0,-30.0,200.0
JC140,chronic implant,2023-12-05 16:00:00,22420007032,250.0,-3893.0,2845.0,125.0,-110.0,0.0
JC140,chronic implant,2023-12-05 16:00:00,22420007231,-4421.0,-2734.0,3696.0,130.0,-35.0,0.0
JC140,chronic implant,2023-12-05 16:00:00,22420007301,-3017.0,1219.0,2514.0,140.0,85.0,5.0


In [4]:
insertions = ProbeInsertion * cp.ChronicInsertion() & 'subject_name = "MM008"'
insertions = insertions.fetch(as_dict=True)

regions_to_show = cp.ChronicInsertion().TargetedRegion() & 'subject_name = "MM008"'
regions_to_show = regions_to_show.fetch('region_acronym','hemisphere', as_dict=True)

In [9]:
plotter = pv.Plotter(notebook=True)
plotter.show()

#from pyvistaqt import BackgroundPlotter
#plotter = BackgroundPlotter()

atlas = Atlas(plotter, min_tree_depth=8, max_tree_depth=8)
for region in regions_to_show:
    atlas.add_atlas_region_mesh(region['region_acronym'])

for i, entrypoint in enumerate(insertions):
    entry_point = [entrypoint['insertion_ml'], entrypoint['insertion_ap']]
    depth = entrypoint['insertion_depth']
    angles = [entrypoint['insertion_el'], entrypoint['insertion_spin'], entrypoint['insertion_az']]
    if entrypoint['holder_id'] == 0:
        prb = NeuropixelsChronicHolder('NP1','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint['holder_id'] == 1:
        prb = NeuropixelsChronicHolder('NP1','freely_moving', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint['holder_id'] == 2:
        prb = NeuropixelsChronicHolder('NP24','head_fixed', plotter, root_intersection_mesh=atlas.meshes['root'])
    elif entrypoint['holder_id'] == 3:
        prb = NeuropixelsChronicHolder('NP24','freely_moving', plotter, root_intersection_mesh=atlas.meshes['root'])

    prb.drive_probe_from_entry(entry_point, angles, depth)
    prb.make_inactive()

plotter.isometric_view()

{'subject_name': 'MM008', 'procedure_type': 'chronic implant', 'procedure_datetime': datetime.datetime(2023, 3, 14, 20, 0), 'probe_id': '19454421152', 'insertion_ap': 1137.0, 'insertion_ml': -1984.0, 'insertion_depth': 3937.0, 'insertion_el': 95.0, 'insertion_az': 70.0, 'insertion_spin': 0.0, 'holder_id': 0}


AttributeError: 'dict' object has no attribute 'probe_type'

In [ ]:
from pathlib import Path
savedir = Path(r'C:\Data\churchland\chronic_manuscript_figs\raw_plots\trajectories')

#plotter.save_graphic(savedir / f'{SUBJECT}_trajectories.pdf', raster=False)
plotter.screenshot(savedir / f'{SUBJECT}_trajectories.png', return_img=False, scale=5)